# Data Cleaning: Exploring Ebay Car Sales Data
The goal of this project is to clean and analyze a dataset of used cars from classifieds section of the German eBay website.

This project is a part of the Dataquest.io Data Scientist course.

## Data Source
The original dataset was scraped and uploaded to Kaggle. The data we will use for this analysis is a subset of 50,000 data points from the full set.  Furthermore, Dataquest has dirtied the sample dataset so that we can practice data-cleaning.

## Data Description
The dataset contains the following attributes:
- **dateCrawled** - When the ad was originally crawled.
- **name** - Name of the car.
- **seller** - Private seller or dealer.
- **offerType** - Type of listing.
- **price** - Advertised price of the car.
- **abtest** - Whether the listing is included in an A/B test.
- **vehicleType** - The vehicle Type.
- **yearOfRegistration** - Year in which the car was first registered.
- **gearbox** - Transmission type.
- **powerPS** - The power of the car measured in PS.
- **model** - Car model name.
- **odometer** - How many kilometers the car has driven.
- **monthOfRegistration** - Month in which the war was first registered.
- **fuelType** - Type of fuel the car uses.
- **brand** - Brand of car.
- **notRepairedDamage** - Whether the car has damage which is not yet repaired.
- **dateCreated** - Date on which the eBay listing was created.
- **nrOfPictures** - Number of pictures in the ad.
- **postalCode** - The postal code for the location of the vehicle.
- **lastSeenOnline** - When the crawler last saw this ad online.

## Import Python libararies and dataset
First let's import the Pandas and NumPy libaries which we will use to manipulate and analyze the data.  Then we will import the CSV dataset as a Pandas Dataframe.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# This cell is commented out so that 'run-all cells' can work
# autos = pd.read_csv("autos.csv")

Here we encounter our first challenge, the code as-is in the cell above returns an encoding error:

>UnicodeDecodeError: 'utf-8' codec can't decode byte 0xdc in position 23: invalid continuation byte

If you want to see for yourself, download this notebook, remove the # and try to run that cell.

The default encoding for pandas.read_csv is UTF-8. Since this does not work, let's try the next two most frequent encodings Latin-1 and Windows-1252.

In [3]:
autos = pd.read_csv("autos.csv", encoding="Latin-1")

When we try "Latin-1" encoding, the code executes without errors.  Let's take a look at the first few rows and some summary info about our dataset.

In [4]:
# By default .head() method displays the first 5 rows.
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [5]:
# Counts of non-null values for each attribute.
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

Some observations:
- We can confirm that there are 50,000 rows of data.
- Most of the columns contain string values ("object" data type in Pandas.)
- However, some columns are missing data (ex. "vehicleType" has 44,905 entries)

## Change column names
It is difficult for us to read the column names as they are currently written in CamelCase. Let's change these to snake_case and give them more descriptive names.

In [6]:
# List of new column names
new_cols = ["date_crawled", "name", "seller", "offer_type",
            "price", "a_b_test", "vehicle_type", "registration_year",
            "gearbox", "power_PS", "model", "odometer", "registration_month",
            "fuel_type", "brand", "unrepaired_damage", "ad_created",
            "nr_of_pictures", "postal_code", "last_seen"]

In [7]:
# Confirm new_cols has same number of column names
len(new_cols)

20

In [8]:
# Assign new column names to our DataFrame
autos.columns = new_cols

In [9]:
# Display changes made
autos.head()

,date_crawled,name,seller,offer_type,price,a_b_test,vehicle_type,registration_year,gearbox,power_PS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


- We changed all the column names with multiple words to snake_case (ex. "dateCrawled" to "date_crawled".
- We renamed a few headings to be more descriptive:
    - "yearOfRegistration" to **"registration_year"**
    - "monthOfRegistration" to **"registration_month"**
    - "notRepairedDamage" to **"unrepaired_damage"**
    - "dateCreated" to **"ad_created"**

## Initial Exploration and Cleaning
Let's further explore the data to determine what else we can clean.

In [10]:
# Display summary statistics for numerical and categorical columns.
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,a_b_test,vehicle_type,registration_year,gearbox,power_PS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-04-02 15:49:30,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


- We can see that **"seller", "offer_type", and "nr_of_pictures"** largely contain only one value. 
    - We shoud drop these columns since they are unlikely to show us anything interesting.
- We should take a closer look at the columns that are missing data.
- We need to convert **"price"** and **"odometer"** from strings to numerical values.
    - We will need to remove any non-numerical characters, then convert these columns to the appropriate data type.

### Remove columns
We can remove these columns in only a couple lines of code. First, we list the names of the unwanted columns. Then, we can use that list as a parameter in the drop method.

In [11]:
# List of columns to remove
drop_cols = ["seller", "offer_type", "nr_of_pictures"]

# Pass the list of columns as a parameter in the drop method
autos = autos.drop(columns = drop_cols)

### Investigate Missing Data
Let's take a closer look at the following columns with missing data:
- **"vehicle_type"**
- **"gearbox"**
- **"model"**
- **"fuel_type"**
- **"unrepaired_damage"**

Below are the proportions of the values in each column.

In [12]:
autos["vehicle_type"].value_counts(normalize=True,dropna=False)

limousine     0.25718
kleinwagen    0.21644
kombi         0.18254
NaN           0.10190
bus           0.08186
cabrio        0.06122
coupe         0.05074
suv           0.03972
andere        0.00840
Name: vehicle_type, dtype: float64

In [13]:
autos["gearbox"].value_counts(normalize=True,dropna=False)

manuell      0.73986
automatik    0.20654
NaN          0.05360
Name: gearbox, dtype: float64

In [14]:
# .head() is used because 245 unique models is too many to display
(autos["model"]
 .value_counts(normalize=True,dropna=False)
 .head()
)

golf      0.08048
andere    0.07056
3er       0.05522
NaN       0.05516
polo      0.03514
Name: model, dtype: float64

In [15]:
autos["fuel_type"].value_counts(normalize=True,dropna=False)

benzin     0.60214
diesel     0.29134
NaN        0.08964
lpg        0.01382
cng        0.00150
hybrid     0.00074
andere     0.00044
elektro    0.00038
Name: fuel_type, dtype: float64

In [16]:
autos["unrepaired_damage"].value_counts(normalize=True,dropna=False)

nein    0.70464
NaN     0.19658
ja      0.09878
Name: unrepaired_damage, dtype: float64

We can see that nearly 20% of **"unrepaired_damage"** entries are missing.

To deal with the missing data, our options are:
- Delete the columns with missing values.
- Delete the rows with missing values.
- Insert values where they are currently missing.
- Leave the missing values alone.

Out of these columns **"gearbox"** has the smallest proportion of missing values. The most frequent value in this column (*"manuell"*) appears much more than the other. Let's use this value to replace those that are missing.

In [17]:
# Replace missing values in "gearbox" column
autos["gearbox"] = autos["gearbox"].fillna("manuell")

In [18]:
# Display value counts
autos["gearbox"].value_counts(normalize=True,dropna=False)

manuell      0.79346
automatik    0.20654
Name: gearbox, dtype: float64

We have successfully filled all the NaN's in the **"gearbox"** column.

### Convert strings to numbers
Now let's convert the entries of **"price"** and **"odometer"**.

First, let's remove the non-numeric characters by using the substring .replace() method:
- The first argument is the character we want to remove.
- The second argument is the replacement for the character. (In our case, literally nothing, indicated by the empty quotes.)

In [19]:
# Remove non-numeric characters
autos["price"] = autos["price"].str.replace("$", "")
autos["price"] = autos["price"].str.replace(",", "")

autos["odometer"] = autos["odometer"].str.replace("km", "")
autos["odometer"] = autos["odometer"].str.replace(",", "")

Next, let's convert these columns to the integer data type.

In [20]:
# Convert data types to int
autos["price"] = autos["price"].astype(int)
autos["odometer"] = autos["odometer"].astype(int)

Finally, let's rename **"odometer"** so that we can keep track of its units.

In [21]:
# Rename "odometer" to describe units
autos = autos.rename(columns={"odometer": "odometer_km"})

## Exploring Price and Odometer Columns
Now that we have transformed the **"price"** and **"odometer_km""** columns, we can evaluate them numerically.

### Price
First let's bring up the stats for **"price"** and look for anything suspicious.

In [22]:
# Summary statistics for "price" column
round(autos["price"].describe())

count       50000.0
mean         9840.0
std        481104.0
min             0.0
25%          1100.0
50%          2950.0
75%          7200.0
max      99999999.0
Name: price, dtype: float64

We can tell that this is a skewed distribution for the following reasons:
- The mean is greatly about the mean *(50%)* and even the 3rd Quartile *(75%)*.
    - More than 75% of the prices are below the mean, that doesn't seem right.
- The max value (99,999,999) is clearly driving up the mean.

In this case, we can tell more about the distribution by looking at the median.

In [23]:
autos["price"].median()

2950.0

In [24]:
# Number of unique entries in "price" column
autos["price"].unique().shape[0]

2357

Out of the 2357 unique values, how many are equal to the max?

Let's start by creating a Boolean variable that compares every value in the "price" column against the max. This will return a Series with values of True or False, corresponding to this evaluation.

In [25]:
# Boolean variable to return rows with max price
max_price = autos["price"] == autos["price"].max()

In [26]:
# Return the number of rows meeting the condition
autos.loc[max_price, "price"].count()

1

We can see there is only one entry that matches the max. Let's look at this row of the data.

In [27]:
autos.loc[max_price, :"price"]

,date_crawled,name,price
39705,2016-03-22 14:58:27,Tausch_gegen_gleichwertiges,99999999


Now, let's use the **Fence Rule** to find all the outliers. These rows are good candidates for elimination.

To use the Fence Rule, we need to calculate three numbers:
- Interquartile Range (IQR)
- Lower and Upper Fence: the lower and upper cutoff points for inclusion.

Formulas:
>IQR = Q3 - Q1

>LF = Q1 - (1.5 * IQR)

>UF = Q3 + (1.5 * IQR)

In [28]:
# Define variables:

# IQR = Q3 - Q1
price_IQR = autos["price"].quantile(.75) - autos["price"].quantile(.25)

# LF = Q1 - (1.5 * IQR)
price_LF = autos["price"].quantile(.25) - (1.5 * price_IQR)

# UF = Q3 + (1.5 * IQR)
price_UF = autos["price"].quantile(.75) + (1.5 * price_IQR)

In [29]:
# Display Fence values
print("LF:", price_LF)
print("UF:", price_UF)

LF: -8050.0
UF: 16350.0


We already know that the minimum price is 0, therefore we know there will be no outliers below the Lower Fence.

In [30]:
# Boolean for outliers above Upper Fence
expensive = (autos["price"] > price_UF) == True

In [31]:
# Quantity of outliers to be discarded
autos.loc[expensive, "price"].count()

3784

In [32]:
# List of indices of the rows with price outliers
price_outliers = autos.loc[expensive, :].index

In [33]:
# Drop price outliers from the DataFrame
autos = autos.drop(index = price_outliers)

### Odemeter
Now let's repeat the search and elimination of outliers from **"odometer_km"**.

In [34]:
round(autos["odometer_km"].describe())

count     46216.0
mean     129603.0
std       36812.0
min        5000.0
25%      125000.0
50%      150000.0
75%      150000.0
max      150000.0
Name: odometer_km, dtype: float64

From these summary stats, we can infer the following:
- We do not need to find the Upper Fence because the mean and Q3 are equal to the max.
- If any outliers exist, we would find them below the Lower Fence.

In [35]:
# Number of unique entries in "odometer_km" column
autos["odometer_km"].unique().shape[0]

13

There are only thirteen unique odometer recordings. Let's find out how many of each.

In [36]:
autos["odometer_km"].value_counts().sort_index(ascending=True)

5000        863
10000       137
20000       460
30000       496
40000       530
50000       717
60000       883
70000       967
80000      1199
90000      1526
100000     1917
125000     4834
150000    31687
Name: odometer_km, dtype: int64

The majority of the entries are equal to the max value of 150,000 km. Should we eliminate any entries?

In [37]:
# Define parameters to classify outliers in "odometer_km"

# IQR = Q3 - Q1
odometer_IQR = autos["odometer_km"].quantile(.75) - autos["odometer_km"].quantile(.25)

# LF = Q1 - (1.5 * IQR)
odometer_LF = autos["odometer_km"].quantile(.25) - (1.5 * odometer_IQR)

print(odometer_LF)

87500.0


Now that we have the Lower Fence value, let's identify and eliminate the outlier rows.

In [38]:
# Boolean variable that returns rows of outliers below the Lower Fence
low_odometer = (autos["odometer_km"] < odometer_LF) == True

# Array of indices of the rows that are outliers
odometer_outliers = autos.loc[low_odometer, :].index

In [39]:
# Drop rows containing outliers from the DataFrame
autos = autos.drop(index = odometer_outliers)

In [40]:
# Evaluate DataFrame after changes
autos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39964 entries, 0 to 49999
Data columns (total 17 columns):
date_crawled          39964 non-null object
name                  39964 non-null object
price                 39964 non-null int64
a_b_test              39964 non-null object
vehicle_type          35698 non-null object
registration_year     39964 non-null int64
gearbox               39964 non-null object
power_PS              39964 non-null int64
model                 37819 non-null object
odometer_km           39964 non-null int64
registration_month    39964 non-null int64
fuel_type             36260 non-null object
brand                 39964 non-null object
unrepaired_damage     31530 non-null object
ad_created            39964 non-null object
postal_code           39964 non-null int64
last_seen             39964 non-null object
dtypes: int64(6), object(11)
memory usage: 5.5+ MB


By removing the **"price"** and **"odometer_km"** outliers, we have eliminated about 10,000 rows from the data.

## Date Columns
The date columns contain date and timestamp values. We will need to isolate just the dates to evaluate the date range for these columns.

In [41]:
date_cols = ["date_crawled", "last_seen", "ad_created"]

In [42]:
autos[date_cols].head(3)

,date_crawled,last_seen,ad_created
0,2016-03-26 17:47:46,2016-04-06 06:45:54,2016-03-26 00:00:00
1,2016-04-04 13:38:56,2016-04-06 14:45:08,2016-04-04 00:00:00
4,2016-04-01 14:38:50,2016-04-01 14:38:50,2016-04-01 00:00:00


Since these columns are stored as strings, we can use substring methods to extract the dates.  We can look at the first ten characters for each: **YYYY-MM-DD**

In [43]:
# Display number of unique dates for each column
print("date_crawled:", autos["date_crawled"].str[:10].unique().shape[0])
print("last_seen:", autos["last_seen"].str[:10].unique().shape[0])
print("ad_created:", autos["ad_created"].str[:10].unique().shape[0])

date_crawled: 34
last_seen: 34
ad_created: 68


Let's check out the distribution of dates in chronological order.

In [44]:
(autos["date_crawled"].str[:10]
 # normalize=True returns the proportion instead of count
 .value_counts(normalize=True)
 # sort the dates from earliest to latest
 .sort_index(ascending=True)
)

2016-03-05    0.024947
2016-03-06    0.014063
2016-03-07    0.036283
2016-03-08    0.034231
2016-03-09    0.033805
2016-03-10    0.032579
2016-03-11    0.032079
2016-03-12    0.037384
2016-03-13    0.015014
2016-03-14    0.037158
2016-03-15    0.034006
2016-03-16    0.029852
2016-03-17    0.031954
2016-03-18    0.013187
2016-03-19    0.033580
2016-03-20    0.037309
2016-03-21    0.037534
2016-03-22    0.032504
2016-03-23    0.032654
2016-03-24    0.029001
2016-03-25    0.032629
2016-03-26    0.032454
2016-03-27    0.030928
2016-03-28    0.034631
2016-03-29    0.033830
2016-03-30    0.033981
2016-03-31    0.031779
2016-04-01    0.033055
2016-04-02    0.034706
2016-04-03    0.038184
2016-04-04    0.036733
2016-04-05    0.013362
2016-04-06    0.003453
2016-04-07    0.001151
Name: date_crawled, dtype: float64

In [45]:
(autos["last_seen"].str[:10]
 # normalize=True returns the proportion instead of count
 .value_counts(normalize=True)
 # sort the dates from earliest to latest
 .sort_index(ascending=True)
)

2016-03-05    0.001126
2016-03-06    0.004804
2016-03-07    0.006005
2016-03-08    0.008558
2016-03-09    0.010560
2016-03-10    0.011210
2016-03-11    0.013787
2016-03-12    0.025598
2016-03-13    0.009609
2016-03-14    0.013412
2016-03-15    0.016590
2016-03-16    0.017441
2016-03-17    0.029777
2016-03-18    0.007507
2016-03-19    0.016890
2016-03-20    0.022020
2016-03-21    0.021770
2016-03-22    0.022370
2016-03-23    0.019442
2016-03-24    0.020844
2016-03-25    0.020218
2016-03-26    0.017666
2016-03-27    0.016890
2016-03-28    0.022045
2016-03-29    0.023046
2016-03-30    0.025448
2016-03-31    0.024597
2016-04-01    0.024222
2016-04-02    0.025673
2016-04-03    0.025623
2016-04-04    0.025323
2016-04-05    0.118457
2016-04-06    0.209063
2016-04-07    0.122410
Name: last_seen, dtype: float64

In [46]:
# Chronological list of dates in "date_crawled" column
date_crawled_range = (
    autos["date_crawled"].str[:10]
    .value_counts()
    .sort_index(ascending=True)
    .index
)

In [47]:
# Chronological list of dates in "last_seen" column
last_seen_range = (
    autos["last_seen"].str[:10]
    .value_counts()
    .sort_index(ascending=True)
    .index
)

In [48]:
# Compare date ranges of crawler
date_crawled_range == last_seen_range

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

The comparison above confirms that there are no errors in range of dates recorded by the crawler.

In [49]:
# Display 10 most frequent dates in "ad_created" column
(
    autos["ad_created"].str[:10]
    .value_counts(normalize=True,ascending=False)
    .head(10)
)

2016-04-03    0.038460
2016-03-21    0.037659
2016-03-20    0.037309
2016-04-04    0.037259
2016-03-12    0.037234
2016-03-14    0.035607
2016-03-07    0.035132
2016-03-28    0.034606
2016-04-02    0.034306
2016-03-08    0.034206
Name: ad_created, dtype: float64

There does not appear to be a date that is greatly more frequent than any other in **"ad_created"**.  The most frequent date 2016-04-03 only accounts for 3.8% of the date range.

## Registration Data
Let's evaluate the **"registration_month"**, and **"registration_year"** columns for any irregulaties that should be cleaned.

In [50]:
# Display counts for month in chronological order
autos["registration_month"].value_counts().sort_index(ascending=True)

0     4313
1     2618
2     2378
3     3974
4     3206
5     3289
6     3488
7     3106
8     2521
9     2670
10    2928
11    2644
12    2829
Name: registration_month, dtype: int64

It is odd that some of the months were recorded as 0.  These could be explained as either:
- Missing values.
- Or, reporting errors by sellers

Let's take a quick look at the summary stats for these rows to see if there is an obvious pattern.

In [51]:
autos.loc[autos["registration_month"] == 0, :].describe(include='all')

,date_crawled,name,price,a_b_test,vehicle_type,registration_year,gearbox,power_PS,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
count,4313,4313,4313.000000,4313,2786,4313.000000,4313,4313.000000,3614,4313.000000,4313.0,2864,4313,1804,4313,4313.000000,4313
unique,4294,4031,NaN,2,8,NaN,2,NaN,183,NaN,NaN,5,40,2,40,NaN,4204
top,2016-03-21 16:37:21,Renault_Twingo,NaN,test,kleinwagen,NaN,manuell,NaN,golf,NaN,NaN,benzin,volkswagen,nein,2016-03-21 00:00:00,NaN,2016-04-05 17:26:26
freq,2,14,NaN,2233,840,NaN,3821,NaN,402,NaN,NaN,2199,1105,1171,168,NaN,3
mean,NaN,NaN,1797.414561,NaN,NaN,2001.641317,NaN,70.355205,NaN,144789.009970,0.0,NaN,NaN,NaN,NaN,48098.472989,NaN
std,NaN,NaN,2395.728697,NaN,NaN,8.978666,NaN,85.998716,NaN,14106.078367,0.0,NaN,NaN,NaN,NaN,25849.241639,NaN
min,NaN,NaN,0.000000,NaN,NaN,1910.000000,NaN,0.000000,NaN,90000.000000,0.0,NaN,NaN,NaN,NaN,1067.000000,NaN
25%,NaN,NaN,350.000000,NaN,NaN,1997.000000,NaN,0.000000,NaN,150000.000000,0.0,NaN,NaN,NaN,NaN,26919.000000,NaN
50%,NaN,NaN,950.000000,NaN,NaN,2000.000000,NaN,65.000000,NaN,150000.000000,0.0,NaN,NaN,NaN,NaN,46238.000000,NaN
75%,NaN,NaN,2200.000000,NaN,NaN,2005.000000,NaN,115.000000,NaN,150000.000000,0.0,NaN,NaN,NaN,NaN,66538.000000,NaN


Upon a quick inspection, we do not see a discernable pattern for these entries. Let's remove thse rows since 0 is not a valid month.

In [52]:
# List indices of rows with month 0
month_zero = autos.loc[autos["registration_month"] == 0, :].index

# Pass the list as the index parameter to drop these rows
autos = autos.drop(index = month_zero)

Let's take a look at **"registration_year"**.

In [53]:
autos["registration_year"].describe()

count    35651.000000
mean      2002.800146
std         37.608276
min       1934.000000
25%       1999.000000
50%       2003.000000
75%       2006.000000
max       9000.000000
Name: registration_year, dtype: float64

In [54]:
# Counts of the five latest registration years
(
    autos["registration_year"]
    .value_counts()
    .sort_index(ascending=False)
    .head()
)

9000      1
2019      1
2018    328
2017    952
2016    840
Name: registration_year, dtype: int64

Seeing as this data was scraped in 2016, there should not be any cars registered after this year in the dataset. We can confidently say that these rows should be removed.

In [55]:
# List indices for rows with registration years after 2016
impossible_year = (
    autos.loc[(autos["registration_year"] > 2016) == True, :]
    .index
)

In [56]:
# Drop offending rows
autos = autos.drop(index = impossible_year)

In [57]:
# Ten most frequent registration years by proportion
(
    autos["registration_year"].value_counts(normalize=True)
    .head(10)
)

1999    0.074573
2000    0.073933
2005    0.071896
2004    0.070354
2003    0.069889
2001    0.067474
2006    0.066077
2002    0.064419
1998    0.058687
2007    0.053217
Name: registration_year, dtype: float64

The ten most frequent registration years are within the two decades preceding 2016. This makes sense, as we would not expect many cars that are much older to still be on the market.

## Exploring Price by Brand
We can use aggregation to filter the rows by brand to see the average price for each. First, let's bring up a quick summary of the **"brand"** column.

In [58]:
autos["brand"].describe()

count          34369
unique            40
top       volkswagen
freq            7467
Name: brand, dtype: object

Instead of evaluating all 40 brands, let's set a threshold of five percent. We will investigate any brands that make up at least 5% of the sample.

Let's create a dictionary that with brand names as keys. The value for each key is the corresponding proportion for that brand.

In [59]:
# Dictionary with brand name keys, values are corresponding proportion
brand_pct = dict(autos["brand"].value_counts(normalize=True))

Then, we can create a list for the brands that are at least 5% of the sample. We iterate over the dictionary and only add brands that meet this condition.

In [60]:
# List for brand names that meet the condition
brands = []

# Iterate over the key, value pairs in the dictionary
for key, value in brand_pct.items():
    
    # Add brand name to the list if the condition is met
    if value >= 0.05:
        brands.append(key[:])

In [61]:
brands

['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford', 'renault']

We can see that, with the exception of Ford and Renault, the most popular brands in this sample are all German. Now that we have narrowed down the brands, let's calculate the average price for each.

In [62]:
# Create dictionary using list of brands to name the keys
avg_price_by_brand = dict.fromkeys(brands)

# Iterate over the list
for name in brands:
    # Filter DataFrame by rows for each brand
    brand_prices = autos.loc[autos["brand"] == name, "price"]
    
    # Compute average price for this brand, rounded to 2 decimal places
    brand_avg_price = round(brand_prices.mean(), 2)
    
    # Assign average price to corresponding dictionary key
    avg_price_by_brand[name] = brand_avg_price

In [63]:
# Return dictionary entries, sorted by avg price in descending order
sorted(avg_price_by_brand.items(), key=lambda x:x[1], reverse=True)

[('bmw', 5574.63),
 ('audi', 5479.94),
 ('mercedes_benz', 5097.99),
 ('volkswagen', 3833.64),
 ('ford', 2529.18),
 ('opel', 2321.8),
 ('renault', 1851.22)]

The three most expensive brands are BMW, Audi, and Mercedes Benz. These are typically considered luxury brands, which could be a factor in their pricing.

## Aggregating Mileage and Price by Brand
Brand alone is not the only determining factor of a car's value. Let's compare the average mileage by brand to see if there is a connection to the price.

We will accomplish this by creating a new DataFrame with:
- Top brand names for row indices.
- Columns for average price and average odometer_km.

Here are our steps to create the DataFrame:
- Create dictionaries with brand names as keys, averages as values.
- Convert these dictionaries to Pandas Series objects.
- Create Pandas DataFrame with each Series as a column.

We already have a dictionary of the popular brands and their average price: *avg_price_by_brand*. We can use this dictionary to create the first Series and column of the DataFrame.

In [64]:
# Use average price dictionary to build Series
avg_price_series = pd.Series(avg_price_by_brand)

In [65]:
# Create Dataframe
price_vs_km = pd.DataFrame(avg_price_series, columns = ["mean_price"])

In [66]:
# Our DataFrame so far
price_vs_km

,mean_price
volkswagen,3833.64
bmw,5574.63
opel,2321.80
mercedes_benz,5097.99
audi,5479.94
ford,2529.18
renault,1851.22


Here is what our DataFrame looks like so far. Let's proceed to generate the next column.

In [67]:
# Dictionary for average "odometer_km" by brand, using brand names as keys
avg_km_by_brand = dict.fromkeys(brands)

# Iterate over the list
for name in brands:
    # Filter DataFrame by rows for each brand
    brand_km = autos.loc[autos["brand"] == name, "odometer_km"]
    
    # Compute average odometer_km for this brand, rounded to nearest integer
    brand_avg_km = round(brand_km.mean())
    
    # Assign average price to corresponding dictionary key
    avg_km_by_brand[name] = brand_avg_km

In [68]:
# Create Series from the avg_km_by_brand dictionary, with values as integers
avg_km_series = pd.Series(avg_km_by_brand, dtype=int)

# Add the Series to the DataFrame as a new column
price_vs_km["mean_odometer_km"] = avg_km_series

# Display the completed DataFrame
price_vs_km

,mean_price,mean_odometer_km
volkswagen,3833.64,143357
bmw,5574.63,144622
opel,2321.80,141694
mercedes_benz,5097.99,144819
audi,5479.94,145722
ford,2529.18,140546
renault,1851.22,140474


Now that our DataFrame is complete, let's evaluate the summary statistics.

In [69]:
price_vs_km["mean_odometer_km"].describe()

count         7.000000
mean     143033.428571
std        2143.866589
min      140474.000000
25%      141120.000000
50%      143357.000000
75%      144720.500000
max      145722.000000
Name: mean_odometer_km, dtype: float64

The low standard deviation indicates that there is not much variation in the average odometer reading. Let's see which brands correspond to the maximum and minimum average odometer.

In [70]:
# Variable to store max of "mean_odometer_km"
max_mean_km = price_vs_km["mean_odometer_km"].max()

# Display row for brand with max
price_vs_km.loc[price_vs_km["mean_odometer_km"] == max_mean_km, :]

,mean_price,mean_odometer_km
audi,5479.94,145722


In [71]:
# Variable to store min of "mean_odometer_km"
min_mean_km = price_vs_km["mean_odometer_km"].min()

# Display row for brand with max
price_vs_km.loc[price_vs_km["mean_odometer_km"] == min_mean_km, :]

,mean_price,mean_odometer_km
renault,1851.22,140474


That's interesting! We can see that the brand with the lowest average odometer, Renault, is also the least expensive. Furthermore, the highest average odometer corresponds to Audi, the second most expensive brand in our sample. There does not appear to be a strong correlation between price and the odometer.

## Translate German Categorical Data
Earlier, we have seen German text appear in the following columns: "vehicle_type", "gearbox", "fuel_type", "unrepaired_damage". These are good candidates for translation. We can exclude the following because they contain proper nouns: "name", "brand", and "model".

Here are the steps we will take to translate the text:
- Identify unique values for each column.
- Create dictionaries with German keys, English translations as values
- Use the dictionaries to map the translations to the DataFrame

### Identify Unique Values

In [72]:
# Create lists of unique values for each column, excluding missing values (NaN)
vehicle_type_unique = list(autos.loc[autos["vehicle_type"].notna() == True, "vehicle_type"].unique())
gearbox_unique = list(autos.loc[autos["gearbox"].notna() == True, "gearbox"].unique())
fuel_type_unique = list(autos.loc[autos["fuel_type"].notna() == True, "fuel_type"].unique())

In [73]:
vehicle_type_unique

['bus', 'limousine', 'kombi', 'kleinwagen', 'coupe', 'suv', 'cabrio', 'andere']

In [74]:
gearbox_unique

['manuell', 'automatik']

In [75]:
fuel_type_unique

['lpg', 'benzin', 'diesel', 'cng', 'hybrid', 'andere', 'elektro']

We can see that some of the words are abbreviations ("lpg", "cng", "suv"). We will capitalize the abbreviations when we translate. Other words are the same in English, so we will leave those alone.

### Create Mapping Dictionaries
When we create these dictionaries, it is important that the keys are exactly as they appear in the data.

In [76]:
# Create dictionaries with translations for each key
vehicle_type_eng = {
    "bus": "bus",
    "limousine": "limousine",
    "kombi": "station wagon",
    "kleinwagen": "small car",
    "coupe": "coupe",
    "suv": "SUV",
    "cabrio": "cabrio",
    "andere": "other"
}

gearbox_eng = {
    "manuell": "manual",
    "automatik": "automatic"
}

fuel_type_eng = {
    "lpg": "LPG",
    "benzin": "gasoline",
    "diesel": "diesel",
    "cng": "CNG",
    "hybrid": "hybrid",
    "andere": "other",
    "elektro": "electro"
}

### Apply Translations to DataFrame
Now that we have the translations, let's set these changes in our DataFrame.

In [77]:
# Apply changes to each column
autos["vehicle_type"] = autos["vehicle_type"].map(vehicle_type_eng)
autos["gearbox"] = autos["gearbox"].map(gearbox_eng)
autos["fuel_type"] = autos["fuel_type"].map(fuel_type_eng)

In [78]:
print(list(autos["vehicle_type"].unique()))
print(list(autos["gearbox"].unique()))
print(list(autos["fuel_type"].unique()))

['bus', 'limousine', 'station wagon', nan, 'small car', 'coupe', 'SUV', 'cabrio', 'other']
['manual', 'automatic']
['LPG', 'gasoline', 'diesel', nan, 'CNG', 'hybrid', 'other', 'electro']


We have succesfully translated these values!

## Conclusion
In summary, here is what we have accomplished in this project:
- Corrected an encoding error while importing a CSV.
- Changed column names.
- Removed columns.
- Filled missing values.
- Converted strings to numbers.
- Identified and removed outliers.
- Performed substring operations.
- Removed rows.
- Aggregated data.
- Created a new DataFrame from aggregate data.
- Replaced values by mapping dictionaries.